In [1]:
import socket
import json
import numpy as np

# Load your trained model
# This should return something that has a .predict() method
# model = load_your_model()  # You implement this

HOST = '127.0.0.1'  # Important: localhost, since it's accessed via SSH tunnel
PORT = 9002

def process_obs(obs_dict):
    scan = obs_dict.get("scan", [])
    odom = obs_dict.get("odom", [])
    obs = np.array(scan + odom).reshape(1, -1)
    return obs

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((HOST, PORT))
s.listen()
print("🟢 RL model server listening on localhost:9001...")

while True:
    conn, addr = s.accept()
    print(conn,addr)
    with conn:
        data = conn.recv(4096)
        print(data)
        if not data:
            continue
        obs_dict = json.loads(data.decode())
        obs = process_obs(obs_dict)
        action = [0,1]#model.predict(obs)[0]  # e.g. [0.2, 0.5]
        response = {
            "linear_x": float(action[0]),
            "angular_z": float(action[1])
        }
        conn.sendall(json.dumps(response).encode())


🟢 RL model server listening on localhost:9001...
<socket.socket fd=57, family=2, type=1, proto=0, laddr=('127.0.0.1', 9001), raddr=('127.0.0.1', 59358)> ('127.0.0.1', 59358)
b'{"scan": [], "odom": []}'
<socket.socket fd=57, family=2, type=1, proto=0, laddr=('127.0.0.1', 9001), raddr=('127.0.0.1', 59364)> ('127.0.0.1', 59364)
b'{"scan": [], "odom": []}'
<socket.socket fd=57, family=2, type=1, proto=0, laddr=('127.0.0.1', 9001), raddr=('127.0.0.1', 59372)> ('127.0.0.1', 59372)
b'{"scan": [], "odom": []}'
<socket.socket fd=57, family=2, type=1, proto=0, laddr=('127.0.0.1', 9001), raddr=('127.0.0.1', 43376)> ('127.0.0.1', 43376)
b'{"scan": [], "odom": []}'
<socket.socket fd=57, family=2, type=1, proto=0, laddr=('127.0.0.1', 9001), raddr=('127.0.0.1', 43386)> ('127.0.0.1', 43386)
b'{"scan": [], "odom": []}'
<socket.socket fd=57, family=2, type=1, proto=0, laddr=('127.0.0.1', 9001), raddr=('127.0.0.1', 43400)> ('127.0.0.1', 43400)
b'{"scan": [], "odom": []}'
<socket.socket fd=57, family=2, t

KeyboardInterrupt: 

In [ ]:
import math

def normalize_angle(angle):
    """Normalize angle to range [-pi, pi]"""
    while angle > math.pi:
        angle -= 2.0 * math.pi
    while angle < -math.pi:
        angle += 2.0 * math.pi
    return angle

def get_velocity_from_action(current_theta, action):
    """
    current_theta: robot's current orientation in radians
    action: list like [1, 0], [-1, 0], [0, 1], [0, -1] representing global direction

    returns: (linear_x, angular_z)
    """

    # Parameters (tune as needed)
    MAX_LINEAR_SPEED = 0.2  # m/s
    MAX_ANGULAR_SPEED = 1.0  # rad/s
    ANGLE_TOLERANCE = 0.1  # radians

    # Target direction in world frame
    target_dir_x, target_dir_y = action

    # Convert target direction to angle
    target_theta = math.atan2(target_dir_y, target_dir_x)

    # Compute difference between current and target heading
    angle_diff = normalize_angle(target_theta - current_theta)

    if abs(angle_diff) < ANGLE_TOLERANCE:
        # Aligned: move forward
        linear_x = MAX_LINEAR_SPEEDively
        angular_z = 0.0
    else:
        # Not aligned: rotate towards target
        linear_x = 0.0
        angular_z = MAX_ANGULAR_SPEED if angle_diff > 0 else -MAX_ANGULAR_SPEED

    return linear_x, angular_z
